In [26]:
#Read in Libraries necessary for cleaning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
import nltk
import seaborn as sns
from spacy.lang.en import English
from langdetect import detect
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
from langdetect import detect_langs

In [27]:
#Read in the dataset
Titles = pd.read_csv('Total_Title_Data.csv')

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_12276\3995532456.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Titles = pd.read_csv('Total_Title_Data.csv')


In [28]:
#Convert to String Type for Text cleaning
Titles = Titles[['EAN', 'Title', 'Alt Vend', 'Ingram Category Code', 'MJR BISAC', 'BISAC', 'Annotation', 'Category']].astype(str)

In [29]:
#Checkout the dataset to audit Dtype
Titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426870 entries, 0 to 426869
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   EAN                   426870 non-null  object
 1   Title                 426870 non-null  object
 2   Alt Vend              426870 non-null  object
 3   Ingram Category Code  426870 non-null  object
 4   MJR BISAC             426870 non-null  object
 5   BISAC                 426870 non-null  object
 6   Annotation            426870 non-null  object
 7   Category              426870 non-null  object
dtypes: object(8)
memory usage: 26.1+ MB


In [30]:
#Read in first 5 rows in the dataset to audit observations
Titles.head()

,EAN,Title,Alt Vend,Ingram Category Code,MJR BISAC,BISAC,Annotation,Category
0,9781803143057,Murder at the Country Club: An absolutely unpu...,6072814,FR,FIC,FIC027110,<p><strong>Kitty Underhay is playing doubles.....,1
1,9798440995017,From Sandpaper To Silk (Book One) ...,6107230,OP,OCC,OCC000000,"This is a very suspenseful, engaging, and rive...",1
2,9798885362184,Jason Lang & the iTunes Scandals: The Real-Lif...,6110949,BA,BIO,BIO000000,<p>Real Life Story About the Gilgo Beach Murde...,1
3,9781636160887,Izzie's Incredible Imagination ...,9269037,CL,JUV,JUV001000,<p>Whether you are floating in space or riding...,1
4,9781803810324,"Zeppelins in the Searchlights, Sisters of Merc...",6027623,BA,BIO,BIO026000,"<p>Born in 1909 in Hull, Bernard recalls how h...",1


In [31]:
#Remove HTML from 'Annotation' category using 'Beautiful Soup' package and save as a new column 'Annotation Text'
from bs4 import BeautifulSoup

def remove_html_tags(text):
    cleantext = BeautifulSoup(text, "lxml").text
    return cleantext

Titles['Annotation Text'] = Titles['Annotation'].apply(remove_html_tags)

C:\Users\Pupnsuds90\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [32]:
#Audit Dataframe
Titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426870 entries, 0 to 426869
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   EAN                   426870 non-null  object
 1   Title                 426870 non-null  object
 2   Alt Vend              426870 non-null  object
 3   Ingram Category Code  426870 non-null  object
 4   MJR BISAC             426870 non-null  object
 5   BISAC                 426870 non-null  object
 6   Annotation            426870 non-null  object
 7   Category              426870 non-null  object
 8   Annotation Text       426870 non-null  object
dtypes: object(9)
memory usage: 29.3+ MB


In [33]:
#Remove 'junk code' titles from the dataset with the pattern below

# Define the regex pattern
pattern = r'&#\d+;'

# Create a new dataset for titles matching the pattern
new_dataset = Titles[Titles['Title'].str.contains(pattern, regex=True)].copy()

# Remove the matching titles from the original dataset
Titles = Titles[~Titles['Title'].str.contains(pattern, regex=True)].reset_index(drop=True)

# Print the new dataset containing titles matching the pattern
print("New Dataset:")
print(new_dataset)

# Print the updated original dataset without titles matching the pattern
print("Updated Original Dataset:")
print(Titles)

New Dataset:
                  EAN                                              Title  \
144     9781685364816  The Song of &#23433;&#29738; An-Chee          ...   
154     9789948817444  &#1571;&#1579;&#1610;&#1585;&#1610;           ...   
162     9781647841553  &#26368;&#20986;&#21517;&#30340;&#19968;&#3000...   
177     9789948817260  &#1587;&#1576;&#1578;&#1575;&#1580;&#1585;&#15...   
188     9789948817482  &#1575;&#1604;&#1576;&#1585;&#1607;&#1575;&#16...   
...               ...                                                ...   
421445  9791188195107  Best Korean Short Stories Collection &#45824;&...   
421447  9791188195282  Best Korean Short Stories Collection 2 &#45824...   
422204  9781958313084  Muslimische Frauen und der Hijab-Schleier: Unt...   
422312  9781956555257  Trace and Color: Learning Collection Ages 3-6 ...   
422628  9781772280333  Navzdy Se Mnou: O Mém U&#268;iteli Rabasovi   ...   

       Alt Vend Ingram Category Code MJR BISAC      BISAC  \
144     92237

In [34]:
#review saved, segmented dataset -- this can be explored later, if needed
new_dataset

,EAN,Title,Alt Vend,Ingram Category Code,MJR BISAC,BISAC,Annotation,Category,Annotation Text
144,9781685364816,The Song of &#23433;&#29738; An-Chee ...,9223758,MU,MUS,MUS000000,<p>Anchee Escapes from the Chinese dynasty pas...,1,Anchee Escapes from the Chinese dynasty past t...
154,9789948817444,&#1571;&#1579;&#1610;&#1585;&#1610; ...,6112990,HE,SEL,SEL027000,&#1604;&#1604;&#1602;&#1605;&#1585; &#1585;&#1...,1,للقمر رمزيةٌ خاصة لفهم أنَّ أرواحَنا لا تفنى ح...
162,9781647841553,&#26368;&#20986;&#21517;&#30340;&#19968;&#3000...,9170656,HU,HUM,HUM003000,<p>&#20940;&#40718;&#24180;&#65292;&#20013;&#2...,1,凌鼎年，中國作家協會會員、世界華文微型小說研究會會長、亞洲微電影學院客座教授，美國紐約商務出...
177,9789948817260,&#1587;&#1576;&#1578;&#1575;&#1580;&#1585;&#15...,6112990,FM,FIC,FIC022060,&#1605;&#1575; &#1586;&#1575;&#1604;&#1578; (&...,1,ما زالت (ليليث) تجوس بين الظِّلال، ما زال (لوس...
188,9789948817482,&#1575;&#1604;&#1576;&#1585;&#1607;&#1575;&#16...,6112990,CL,JNF,JNF049000,&#1585;&#1581;&#1604;&#1577; &#1605;&#1606;&#1...,1,رحلة منطقية مِن البداية، مِن أصول المنطق والتف...
...,...,...,...,...,...,...,...,...,...
421445,9791188195107,Best Korean Short Stories Collection &#45824;&...,9037057,LF,FOR,FOR015000,<p>Meet the very finest of Korean short storie...,3,Meet the very finest of Korean short stories. ...
421447,9791188195282,Best Korean Short Stories Collection 2 &#45824...,9037057,LF,FOR,FOR015000,<p>Meet the very finest of Korean short storie...,3,Meet the very finest of Korean short stories. ...
422204,9781958313084,Muslimische Frauen und der Hijab-Schleier: Unt...,9184014,RS,REL,REL037000,<p>In den Medien und in der Gesellschaft wird ...,3,In den Medien und in der Gesellschaft wird die...
422312,9781956555257,Trace and Color: Learning Collection Ages 3-6 ...,9242563,CL,JNF,JNF006020,<p><strong>Perfect tool for home schooling and...,3,Perfect tool for home schooling and having fun...


In [35]:
#Remove 'middle' category, representing 'uncertain' classifications
Titles_Binary = Titles[Titles['Category'].isin(['1','3'])]

In [36]:
#Save selected columns back to Titles_Binary
Titles_Binary = Titles_Binary[['EAN', 'Title', 'Ingram Category Code', 'MJR BISAC', 'BISAC', 'Annotation Text', 'Category']]

In [37]:
#Drop rows containing Null values from the dataset
Titles_Binary.dropna(inplace=False)

,EAN,Title,Ingram Category Code,MJR BISAC,BISAC,Annotation Text,Category
0,9781803143057,Murder at the Country Club: An absolutely unpu...,FR,FIC,FIC027110,Kitty Underhay is playing doubles... with deat...,1
1,9798440995017,From Sandpaper To Silk (Book One) ...,OP,OCC,OCC000000,"This is a very suspenseful, engaging, and rive...",1
2,9798885362184,Jason Lang & the iTunes Scandals: The Real-Lif...,BA,BIO,BIO000000,Real Life Story About the Gilgo Beach Murders ...,1
3,9781636160887,Izzie's Incredible Imagination ...,CL,JUV,JUV001000,Whether you are floating in space or riding a ...,1
4,9781803810324,"Zeppelins in the Searchlights, Sisters of Merc...",BA,BIO,BIO026000,"Born in 1909 in Hull, Bernard recalls how he s...",1
...,...,...,...,...,...,...,...
421883,9780228889755,Zoe and the I Can't Monkey ...,CL,JUV,JUV039140,When Zoe decides to take the training wheels o...,3
421884,9780228889762,Zoe and the I Can't Monkey ...,CL,JUV,JUV039140,When Zoe decides to take the training wheels o...,3
421885,9798215456781,Zombie Apocalypse Call Center ...,FC,FIC,FIC002000,"Who are you going to call, to survive the zomb...",3
421886,9781837873371,Zone Diet: The Most Effective Weight Loss Tool...,CW,CKB,CKB000000,"This book analyzes the blue zone diet, includi...",3


In [38]:
#Identify Language by Annotation Text in the dataset using Python package 'detect_langs'

#Define the Function 'detect_language'
def detect_language(text):
    try:
        languages = detect_langs(text)
        return languages[0].lang
    except:
        return None

# Apply the detect_language function to the 'text_column' and create a new column 'language_detected'
Titles_Binary['language_detected'] = Titles_Binary['Annotation Text'].apply(lambda x: detect_language(x))

# Print the updated DataFrame
Titles_Binary

,EAN,Title,Ingram Category Code,MJR BISAC,BISAC,Annotation Text,Category,language_detected
0,9781803143057,Murder at the Country Club: An absolutely unpu...,FR,FIC,FIC027110,Kitty Underhay is playing doubles... with deat...,1,en
1,9798440995017,From Sandpaper To Silk (Book One) ...,OP,OCC,OCC000000,"This is a very suspenseful, engaging, and rive...",1,en
2,9798885362184,Jason Lang & the iTunes Scandals: The Real-Lif...,BA,BIO,BIO000000,Real Life Story About the Gilgo Beach Murders ...,1,en
3,9781636160887,Izzie's Incredible Imagination ...,CL,JUV,JUV001000,Whether you are floating in space or riding a ...,1,en
4,9781803810324,"Zeppelins in the Searchlights, Sisters of Merc...",BA,BIO,BIO026000,"Born in 1909 in Hull, Bernard recalls how he s...",1,en
...,...,...,...,...,...,...,...,...
421883,9780228889755,Zoe and the I Can't Monkey ...,CL,JUV,JUV039140,When Zoe decides to take the training wheels o...,3,en
421884,9780228889762,Zoe and the I Can't Monkey ...,CL,JUV,JUV039140,When Zoe decides to take the training wheels o...,3,en
421885,9798215456781,Zombie Apocalypse Call Center ...,FC,FIC,FIC002000,"Who are you going to call, to survive the zomb...",3,en
421886,9781837873371,Zone Diet: The Most Effective Weight Loss Tool...,CW,CKB,CKB000000,"This book analyzes the blue zone diet, includi...",3,en


In [39]:
#Review the count of languages present
Titles_Binary['language_detected'].value_counts()

en       300359
fr         6954
es         5932
de         4520
it         1776
pt         1162
nl          417
sv          204
fi          162
af          128
id          123
da          110
hr           80
hu           76
hi           67
no           65
sq           56
vi           51
sw           50
et           44
ca           40
tl           36
cy           32
ta           29
pl           24
so           23
tr           22
ru           21
mr           20
cs           13
ro           12
sl           12
te           11
ne           11
ar            9
ja            9
el            7
uk            6
ko            5
zh-cn         4
mk            4
ml            3
sk            3
ur            3
he            3
bn            2
fa            2
bg            1
lv            1
kn            1
lt            1
Name: language_detected, dtype: int64

In [40]:
#Segment out non-English titles for greater interpretability of classifications and modeling decisions
Titles_English = Titles_Binary[Titles_Binary['language_detected'] == 'en']
All_Titles = Titles_Binary

In [41]:
#Rename the observations in the 'Category' so that they are binary 1,0 values
All_Titles['Category'] = All_Titles['Category'].replace('3', '0')
Titles_English['Category'] = Titles_English['Category'].replace('3', '0')

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_12276\2084242066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Titles_English['Category'] = Titles_English['Category'].replace('3', '0')


In [42]:
#Save Datasets with corresponding attribute columns
Titles_English = Titles_English[['EAN', 'Title', 'Ingram Category Code', 'MJR BISAC', 'BISAC', 'Annotation Text', 'Category', 'language_detected']].astype('str')
All_Titles = All_Titles[['EAN', 'Title', 'Ingram Category Code', 'MJR BISAC', 'BISAC', 'Annotation Text', 'Category', 'language_detected']].astype('str')

In [43]:
#Save Datasets to csv for further EDA exploration in R
Titles_English.to_csv('Titles_English.csv')
All_Titles.to_csv('All_Titles.csv')